# Convert ascii to pytables

## prepare conversion of ascii tables into pytables

- author : Sylvie Dagoret-Campagne
- affilication : IJCLab/IN2P3/CNRS
- Creation date : January 22th 2022

## Imports

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import sys,os
sys.path.append('../')
from delight.io import *
from delight.utils import *
from delight.photoz_gp import PhotozGP

In [2]:
from matplotlib.colors import LogNorm

In [3]:
import logging
import coloredlogs
logger = logging.getLogger(__name__)
coloredlogs.install(level='DEBUG', logger=logger,fmt='%(asctime)s,%(msecs)03d %(programname)s %(name)s[%(process)d] %(levelname)s %(message)s')

In [4]:
print(sys.executable)
print(sys.version)
#print(sys.version_info)

/Users/dagoret/anaconda3/bin/python
3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]


In [5]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from delight.io import *

In [7]:
import tables as tb

In [8]:
!ls tmp

data                  parametersTest_14.cfg parametersTest_21.cfg
delight_data          parametersTest_15.cfg parametersTest_3.cfg
parametersTest.cfg    parametersTest_16.cfg parametersTest_4.cfg
parametersTest_1.cfg  parametersTest_17.cfg parametersTest_5.cfg
parametersTest_10.cfg parametersTest_18.cfg parametersTest_6.cfg
parametersTest_11.cfg parametersTest_19.cfg parametersTest_7.cfg
parametersTest_12.cfg parametersTest_2.cfg  parametersTest_8.cfg
parametersTest_13.cfg parametersTest_20.cfg parametersTest_9.cfg


## configuration

In [9]:
pwd

'/Users/dagoret/MacOSX/GitHub/LSST/Delight/desc-dc2'

In [10]:
# config file
configdir          = "./tmp"
configfilename     = "parametersTest.cfg"
configfullfilename = os.path.join(configdir,configfilename)
# data
workdir               = "./tmp/delight_data"
filename_galaxies     = "galaxies-fluxredshifts.txt"
fullfilename_galaxies = os.path.join(workdir,filename_galaxies)

In [11]:
paramFileName = configfullfilename
params        = parseParamFile(paramFileName, verbose=False)

# Delight Parameters

In [12]:
DL = approx_DL()
redshiftDistGrid, redshiftGrid, redshiftGridGP = createGrids(params)
numZ = redshiftGrid.size

# Locate which columns of the catalog correspond to which bands.

bandIndices, bandNames, bandColumns, bandVarColumns, redshiftColumn,refBandColumn = readColumnPositions(params, prefix="target_")

In [13]:
print(bandIndices)

[0 1 2 3 4 5]


In [14]:
print(bandNames)

['DC2LSST_u' 'DC2LSST_g' 'DC2LSST_r' 'DC2LSST_i' 'DC2LSST_z' 'DC2LSST_y']


In [15]:
print(bandColumns)

[ 0  2  4  6  8 10]


In [16]:
print(bandVarColumns)

[ 1  3  5  7  9 11]


In [17]:
print(redshiftColumn)

12


In [18]:
print(refBandColumn)

4


# Training

In [19]:
nbcol_train = len(params['training_bandOrder'])
nbcol_train

13

In [20]:
numThreads = 1
threadNum = 0

numObjectsTraining = np.sum(1 for line in open(params['training_catFile']))

msg= 'Number of Training Objects ' + str(numObjectsTraining)
logger.info(msg)

firstLine = int(threadNum * numObjectsTraining / numThreads)
lastLine = int(min(numObjectsTraining,(threadNum + 1) * numObjectsTraining / numThreads))
numLines = lastLine - firstLine

msg ='Thread ' +  str(threadNum) + ' , analyzes lines ' + str(firstLine) + ' , to ' + str(lastLine)
logger.info(msg)

<ipython-input-20-181498cf5da6>:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  numObjectsTraining = np.sum(1 for line in open(params['training_catFile']))
2022-01-23 15:56:11,843 ipykernel_launcher.py __main__[87906] INFO Number of Training Objects 3755
2022-01-23 15:56:11,844 ipykernel_launcher.py __main__[87906] INFO Thread 0 , analyzes lines 0 , to 3755


In [21]:
crossValidate_flag = params['training_crossValidate']
crossValidate_flag

False

In [22]:
trainingDataIter1 = getDataFromFile(params, firstLine, lastLine,prefix="training_", getXY=True,CV=crossValidate_flag)
loc = 0
for z, normedRefFlux,bands, fluxes, fluxesVar,bandsCV, fluxesCV, fluxesVarCV,X, Y, Yvar in trainingDataIter1:
    print(loc,"\t z",z,"\t Nflux",normedRefFlux,"\t bands",bands,"\t fluxes",fluxes,"\t fluxesVar",fluxesVar,
          "\t bandsCV",bandsCV,"\t fluxesCV", fluxesCV,"\t fluxesVarCV", fluxesVarCV,"\t X",X,"\t Y",Y,"\t Yvar",Yvar)
    
    if loc>2:
        break
    loc += 1

0 	 z 0.02043498739444738 	 Nflux 3031650.7320414316 	 bands [0 1 2 3 4 5] 	 fluxes [1349.60090537 3647.50826794 4841.59244168 5544.0759444  5751.77741451
 5981.23969488] 	 fluxesVar [ 46.38848211 332.9087944  586.39517919 768.95409146 827.86598966
 895.94544786] 	 bandsCV None 	 fluxesCV None 	 fluxesVarCV None 	 X [[0.00000000e+00 2.04349874e-02 2.60222573e+11]
 [1.00000000e+00 2.04349874e-02 2.60222573e+11]
 [2.00000000e+00 2.04349874e-02 2.60222573e+11]
 [3.00000000e+00 2.04349874e-02 2.60222573e+11]
 [4.00000000e+00 2.04349874e-02 2.60222573e+11]
 [5.00000000e+00 2.04349874e-02 2.60222573e+11]] 	 Y [[1349.60090537]
 [3647.50826794]
 [4841.59244168]
 [5544.0759444 ]
 [5751.77741451]
 [5981.23969488]] 	 Yvar [[ 46.38848211]
 [332.9087944 ]
 [586.39517919]
 [768.95409146]
 [827.86598966]
 [895.94544786]]
1 	 z 0.01936132305993654 	 Nflux 81129.16941288905 	 bands [0 1 2 3 4 5] 	 fluxes [ 50.13226396 115.50270113 135.77265951 148.36348784 152.33461248
 155.26331547] 	 fluxesVar [0.229

## Create a pytable

- https://www.pytables.org/usersguide/introduction.html

In [23]:
# data
workdir               = "./tmp/delight_data"
filename_galaxies_h5     = "galaxies-fluxredshifts.h5"
fullfilename_galaxies_h5 = os.path.join(workdir,filename_galaxies_h5)

In [24]:
# Open a file in "w"rite mode
fileh = tb.open_file(fullfilename_galaxies_h5, mode = "w")

In [25]:
# Get the HDF5 root group
root = fileh.root

In [26]:
#Create the groups
group = fileh.create_group(root, "training_catFile","training galaxy catalog")

### data structure

In [27]:
class Galaxy(tb.IsDescription):
    idnumber       = tb.Int64Col()      # Signed 64-bit integer
    redshift       = tb.Float32Col()    # float32 redshift
    normedRedfFlux = tb.Float32Col()    # float32 normalized Flux
    bands          = tb.Int32Col(shape = (len(bandNames),)) 
    fluxes         = tb.Float32Col(shape = (len(bandNames),)) 
    fluxesVar      = tb.Float32Col(shape = (len(bandNames),)) 
    if crossValidate_flag:
        bandsCV        = tb.Int32Col(shape = (len(bandNames),)) 
        fluxesCV       = tb.Float32Col(shape = (len(bandNames),)) 
        fluxesCVVar    = tb.Float32Col(shape = (len(bandNames),)) 
    X              = tb.Float32Col(shape = (len(bandNames),3))   # Gaussian Process X
    Y              = tb.Float32Col(shape = (len(bandNames),1))   # Gaussian Process Y
    Yvar           = tb.Float32Col(shape = (len(bandNames),1))   # Gaussian Process Yvar

In [28]:
# Create 1 table in group
table = fileh.create_table(group, "galaxies", Galaxy)

In [29]:
# Get the record object associated with the table:
row = table.row

In [30]:
trainingDataIter1 = getDataFromFile(params, firstLine, lastLine,prefix="training_", getXY=True,CV=crossValidate_flag)
loc = 0
for z, normedRefFlux,bands, fluxes, fluxesVar,bandsCV, fluxesCV, fluxesVarCV,X, Y, Yvar in trainingDataIter1:
       
    row['idnumber']       = loc
    row['redshift']       = z
    row['normedRedfFlux'] = normedRefFlux
    row['bands']          = bands
    row['fluxes']         = fluxes
    row['fluxesVar']      = fluxesVar
    if crossValidate_flag:
        row['bandsCV']        = bandsCV
        row['fluxesCV']       = fluxsCV
        row['fluxesVarCV']    = fluxsVarCV
    row['X']              = X
    row['Y']              = Y
    row['Yvar']           = Yvar
        # This injects the Record values
    row.append()
    loc += 1
    
 # Flush the table buffers
table.flush()

In [31]:
table

/training_catFile/galaxies (Table(3755,)) ''
  description := {
  "X": Float32Col(shape=(6, 3), dflt=0.0, pos=0),
  "Y": Float32Col(shape=(6, 1), dflt=0.0, pos=1),
  "Yvar": Float32Col(shape=(6, 1), dflt=0.0, pos=2),
  "bands": Int32Col(shape=(6,), dflt=0, pos=3),
  "fluxes": Float32Col(shape=(6,), dflt=0.0, pos=4),
  "fluxesVar": Float32Col(shape=(6,), dflt=0.0, pos=5),
  "idnumber": Int64Col(shape=(), dflt=0, pos=6),
  "normedRedfFlux": Float32Col(shape=(), dflt=0.0, pos=7),
  "redshift": Float32Col(shape=(), dflt=0.0, pos=8)}
  byteorder := 'little'
  chunkshape := (315,)

In [32]:
# Finally, close the file (this also will flush all the remaining buffers!)
fileh.close()    

# Read the HDF5 file

In [33]:
# Open a file in "w"rite mode
fileh5 = tb.open_file(fullfilename_galaxies_h5, mode = "r")

In [34]:
fileh5 

File(filename=./tmp/delight_data/galaxies-fluxredshifts.h5, title='', mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/training_catFile (Group) 'training galaxy catalog'
/training_catFile/galaxies (Table(3755,)) ''
  description := {
  "X": Float32Col(shape=(6, 3), dflt=0.0, pos=0),
  "Y": Float32Col(shape=(6, 1), dflt=0.0, pos=1),
  "Yvar": Float32Col(shape=(6, 1), dflt=0.0, pos=2),
  "bands": Int32Col(shape=(6,), dflt=0, pos=3),
  "fluxes": Float32Col(shape=(6,), dflt=0.0, pos=4),
  "fluxesVar": Float32Col(shape=(6,), dflt=0.0, pos=5),
  "idnumber": Int64Col(shape=(), dflt=0, pos=6),
  "normedRedfFlux": Float32Col(shape=(), dflt=0.0, pos=7),
  "redshift": Float32Col(shape=(), dflt=0.0, pos=8)}
  byteorder := 'little'
  chunkshape := (315,)

In [35]:
print(fileh5)

./tmp/delight_data/galaxies-fluxredshifts.h5 (File) ''
Last modif.: 'Sun Jan 23 15:56:12 2022'
Object Tree: 
/ (RootGroup) ''
/training_catFile (Group) 'training galaxy catalog'
/training_catFile/galaxies (Table(3755,)) ''



In [36]:
for node in fileh5:

    print(node)

/ (RootGroup) ''
/training_catFile (Group) 'training galaxy catalog'
/training_catFile/galaxies (Table(3755,)) ''


In [37]:
for leaf in fileh5.root.training_catFile._f_walknodes('Leaf'):

    print(leaf)

/training_catFile/galaxies (Table(3755,)) ''


In [38]:
table = fileh5.root.training_catFile.galaxies

In [39]:
table

/training_catFile/galaxies (Table(3755,)) ''
  description := {
  "X": Float32Col(shape=(6, 3), dflt=0.0, pos=0),
  "Y": Float32Col(shape=(6, 1), dflt=0.0, pos=1),
  "Yvar": Float32Col(shape=(6, 1), dflt=0.0, pos=2),
  "bands": Int32Col(shape=(6,), dflt=0, pos=3),
  "fluxes": Float32Col(shape=(6,), dflt=0.0, pos=4),
  "fluxesVar": Float32Col(shape=(6,), dflt=0.0, pos=5),
  "idnumber": Int64Col(shape=(), dflt=0, pos=6),
  "normedRedfFlux": Float32Col(shape=(), dflt=0.0, pos=7),
  "redshift": Float32Col(shape=(), dflt=0.0, pos=8)}
  byteorder := 'little'
  chunkshape := (315,)

In [40]:
idx = 0
for r in table.iterrows():
    print(r['idnumber'], r['redshift'], r["normedRedfFlux"],r['bands'], r['fluxes'], r['fluxesVar'])
    idx+=1
    
    if idx>5:
        break

0 0.020434986799955368 3031650.75 [0 1 2 3 4 5] [1349.601  3647.5083 4841.5923 5544.076  5751.7773 5981.2397] [ 46.38848 332.90878 586.3952  768.9541  827.86597 895.94543]
1 0.01936132274568081 81129.171875 [0 1 2 3 4 5] [ 50.132263 115.5027   135.77266  148.3635   152.33461  155.26332 ] [0.22932214 0.34490785 0.46987963 0.5672234  0.6261012  0.8124247 ]
2 0.03672067075967789 228639.40625 [0 1 2 3 4 5] [ 40.32481 148.43153 290.04608 418.12015 582.0878  688.9323 ] [ 0.20209084  0.5637308   2.1169982   4.399594    8.5516815  12.171332  ]
3 0.03946906328201294 432717.96875 [0 1 2 3 4 5] [226.8571  495.3165  668.28143 791.3251  846.67633 900.68335] [ 1.5444628  6.1640058 11.192628  15.702942  18.025972  20.626019 ]
4 0.02699350193142891 61875.7265625 [0 1 2 3 4 5] [ 26.815632  70.67079   95.85882  113.154106 116.47627  122.56071 ] [0.17245781 0.13417406 0.23758131 0.335561   0.38236353 0.5795965 ]
5 0.04294832795858383 13431.1494140625 [0 1 2 3 4 5] [ 2.633927  9.400407 17.415903 24.56197 

In [41]:
# Finally, close the file (this also will flush all the remaining buffers!)
fileh.close()    